# **Week 3 Census Data Exploration**
### **Jia Ni**
In this assignment, I conduct a series of analyses on the 2023 race data for all census tracts in LA County: [ACS Demographic and Housing Estimates](https://data.census.gov/table/ACSDP5Y2023.DP05?g=050XX00US06037$1400000&moe=false). Due to RAM capacity limitations, I divided this assignment into four notebooks. In this specific notebook, I visualized the population distribution of different racial groups in LA County on the base map.

### **Import the libraries**

In [1]:
import pandas as pd
import geopandas as gpd
import folium

### **Read and add GeoJSON file to notebook**

In [2]:
tracts_race = gpd.read_file("data/tracts_race.geojson")
tracts_race.head()

,FIPS,White,Black or African American,American Indian and Alaska Native,Asian,Native Hawaiian and Other Pacific Islander,Some Other Race,Two or More Races,White_Percent,Black or African American_Percent,American Indian and Alaska Native_Percent,Asian_Percent,Native Hawaiian and Other Pacific Islander_Percent,Some Other Race_Percent,Two or More Races_Percent,geometry
0,06037101110,2367,146,4,292,5,558,780,57,3.5,0.1,7,0.1,13.4,18.8,"POLYGON ((-118.29418 34.25595, -118.29434 34.2..."
1,06037101122,3047,25,24,544,0,277,281,72.6,0.6,0.6,13,0,6.6,6.7,"POLYGON ((-118.29567 34.26321, -118.29602 34.2..."
2,06037101220,1704,10,87,490,0,517,626,49.6,0.3,2.5,14.3,0,15.1,18.2,"POLYGON ((-118.2842 34.24792, -118.28592 34.24..."
3,06037101221,1835,150,27,390,0,805,724,46.7,3.8,0.7,9.9,0,20.5,18.4,"POLYGON ((-118.28862 34.2523, -118.2898 34.252..."
4,06037101222,1417,14,80,51,25,55,930,55.1,0.5,3.1,2,1,2.1,36.2,"POLYGON ((-118.28968 34.24908, -118.28983 34.2..."


### **Ensure compatibility with mapping libraries**

In [3]:
if tracts_race.crs.to_epsg() != 4326:
    tracts_race = tracts_race.to_crs(epsg=4326)

### **Find the center of the spatial extent**

In [4]:
bounds = tracts_race.total_bounds
minx, miny, maxx, maxy = bounds
center_lat = (miny + maxy) / 2
center_lon = (minx + maxx) / 2

### **Prepare the data and set up the base map for further visualization**

In [5]:
# Convert the White column to float type
tracts_race["White"] = tracts_race["White"].astype(float)
print(tracts_race["White"].dtype)

float64


In [6]:
# Count the number of tracts with no White population
print((tracts_race["White"] == 0).sum())

26


In [7]:
# Split the data into two groups
null_tracts_w = tracts_race[tracts_race["White"] == 0]
nonnull_tracts_w = tracts_race[tracts_race["White"] != 0]

In [8]:
# Set up the base map
m_w = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_w.fit_bounds([[miny, minx], [maxy, maxx]])

### **Create a layered map of the White population**

In [9]:
# Layer 1: Gray-shaded tracts with no White population
folium.GeoJson(
    null_tracts_w,
    name="Non White Population",
    style_function=lambda feature: {
        "fillColor": "gray",
        "color": "white",
        "weight": 0,
        "fillOpacity": 0.7
    }
).add_to(m_w)

In [10]:
# Layer 2: A choropleth map for tracts with a non-zero White population
folium.Choropleth(
    geo_data=nonnull_tracts_w,
    name="White Population",
    data=nonnull_tracts_w,
    columns=["FIPS", "White"],
    key_on="feature.properties.FIPS",
    fill_color="Blues",
    fill_opacity=0.7,
    line_opacity=0,
    legend_name="White Population",
    nan_fill_color="transparent"
).add_to(m_w)

In [11]:
# Layer 3: Census tract borders with tooltips showing detailed information
folium.GeoJson(
    tracts_race,
    name="Borders",
    style_function=lambda feature: {
        "fillOpacity": 0,
        "color": "white",
        "weight": 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=["FIPS", "White"],
                                   aliases=["Census Tract ID", "White Population"],
                                   localize=True)
).add_to(m_w)

In [12]:
# Add the layer control and show the map
folium.LayerControl().add_to(m_w)

In [ ]:
m_w

### **Create a layered map of the Black or African American population**

In [13]:
# Set up the base map
m_b = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_b.fit_bounds([[miny, minx], [maxy, maxx]])

In [14]:
# Convert the Black or African American column to float type
tracts_race["Black or African American"] = tracts_race["Black or African American"].astype(float)

# Count the number of tracts where with no Black or African American population
print((tracts_race["Black or African American"] == 0).sum())

188


In [15]:
# Split the data into two groups
null_tracts_b = tracts_race[tracts_race["Black or African American"] == 0]
nonnull_tracts_b = tracts_race[tracts_race["Black or African American"] != 0]

In [16]:
# Layer 1: Gray-shaded tracts with no Black or African American population
folium.GeoJson(
    null_tracts_b,
    name="Non Black or African American Population",
    style_function=lambda feature: {
        "fillColor": "gray",
        "color": "white",
        "weight": 0,
        "fillOpacity": 0.7
    }
).add_to(m_b)

# Layer 2: A choropleth map for tracts with a non-zero Black or African American population
folium.Choropleth(
    geo_data=nonnull_tracts_b,
    name="Black or African American Population",
    data=nonnull_tracts_b,
    columns=["FIPS", "Black or African American"],
    key_on="feature.properties.FIPS",
    fill_color="Greens",
    fill_opacity=0.7,
    line_opacity=0,
    legend_name="Black or African American Population",
    nan_fill_color="transparent"
).add_to(m_b)

# Layer 3: Census tract borders with tooltips showing detailed information
folium.GeoJson(
    tracts_race,
    name="Borders",
    style_function=lambda feature: {
        "fillOpacity": 0,
        "color": "white",
        "weight": 0.3
    },
    tooltip=folium.GeoJsonTooltip(
        fields=["FIPS", "Black or African American"],
        aliases=["Census Tract ID", "Black or African American Population"],
        localize=True)
).add_to(m_b)

# Add the layer control and show the map
folium.LayerControl().add_to(m_b)

In [ ]:
m_b

### **Create a layered map of the American Indian and Alaska Native population**

In [17]:
# Set up the base map
m_i = folium.Map(location=[center_lat, center_lon], tiles="cartodb positron")
m_i.fit_bounds([[miny, minx], [maxy, maxx]])

In [18]:
# Convert the White column to float type
tracts_race["American Indian and Alaska Native"] = tracts_race["American Indian and Alaska Native"].astype(float)

# Count the number of tracts where with no American Indian and Alaska Native population
print((tracts_race["American Indian and Alaska Native"] == 0).sum())

724


In [19]:
# Split the data into two groups
null_tracts_i = tracts_race[tracts_race["American Indian and Alaska Native"] == 0]
nonnull_tracts_i = tracts_race[tracts_race["American Indian and Alaska Native"] != 0]

In [20]:
# Layer 1: Gray-shaded tracts with no American Indian and Alaska Native population
folium.GeoJson(
    null_tracts_i,
    name="Non American Indian and Alaska Native",
    style_function=lambda feature: {
        "fillColor": "gray",
        "color": "white",
        "weight": 0,
        "fillOpacity": 0.7
    }
).add_to(m_i)

# Layer 2: A choropleth map for tracts with a non-zero American Indian and Alaska Native population
folium.Choropleth(
    geo_data=nonnull_tracts_i,
    name="American Indian and Alaska Native Population",
    data=nonnull_tracts_i,
    columns=["FIPS", "American Indian and Alaska Native"],
    key_on="feature.properties.FIPS",
    fill_color="Purples",
    fill_opacity=0.7,
    line_opacity=0,
    legend_name="American Indian and Alaska Native",
    nan_fill_color="transparent"
).add_to(m_i)

# Layer 3: Census tract borders with tooltips showing detailed information
folium.GeoJson(
    tracts_race,
    name="Borders",
    style_function=lambda feature: {
        "fillOpacity": 0,
        "color": "white",
        "weight": 0.3
    },
    tooltip=folium.GeoJsonTooltip(fields=["FIPS", "American Indian and Alaska Native"],
                                  aliases=["Census Tract ID", "American Indian and Alaska Native Population"],
                                  localize=True)
).add_to(m_i)

# Add the layer control and show the map
folium.LayerControl().add_to(m_i)

In [ ]:
m_i

In [21]:
# Save the maps to HTMLs
m_w.save('White Population.html')
m_b.save('Black or African American Population.html')
m_i.save('American Indian and Alaska Native Population.html')